In [1]:
!pip install -q -U peft bitsandbytes

In [9]:
import warnings
warnings.filterwarnings("ignore")

## PEFT Config

In [2]:
from peft import PeftConfig

peft_model_id = "mehmettozlu/mistral-7b-dolly-fine-tuned"

config = PeftConfig.from_pretrained(peft_model_id)

adapter_config.json:   0%|          | 0.00/713 [00:00<?, ?B/s]

### Model Loading

In [3]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    load_in_4bit=True,
    device_map="auto",
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### Loading Tokenizer

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    padding_size="right",
    add_eos_token=True,
)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Loading the PEFT Model

Orijinal model ile fine-tune ettigimiz modeli birlestirelim

In [5]:
from peft import PeftModel

fine_tune_model = PeftModel.from_pretrained(
    model, peft_model_id
)

adapter_model.safetensors:   0%|          | 0.00/609M [00:00<?, ?B/s]

### Inference with Pipeline

In [6]:
from transformers import pipeline, logging

logging.set_verbosity(logging.CRITICAL)

2024-03-17 12:24:00.530923: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 12:24:00.531081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 12:24:00.710156: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
prompt = """
What is a Python? Here is some context: Python is a high-level, general-purpose programming language.
"""

pipe = pipeline(
    task="text-generation",
    model=fine_tune_model,
    tokenizer=tokenizer,
    eos_token_id=model.config.eos_token_id,
    max_new_tokens=100,
)

result = pipe(f"<s>[INST] {prompt} [/INST]")
generated = result[0]["generated_text"]
print(generated[generated.find("[/INST]")+8:])

Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.

Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including object-oriented, imperative and functional programming or procedural styles. It features a standard library that
